In [1]:
import pandas as pd
import dwsemear
import datetime
import collections

import postgresemear as sql  # mover os arquivos para a pasta principal

In [2]:
dados = pd.read_csv('liquidacoes_012019_062019.csv', sep=',', encoding='latin-1', dtype=str)

In [3]:
dados = dwsemear.limpa_nome_das_colunas(dados)
dados = dwsemear.coluna_cpf_para_inteiro(dados, 'CPF CNPJ')
dados.loc[:,'Dias'] = dados.loc[:,'Dias'].astype(int)
dados.loc[:,'Parcela'] = dados.loc[:,'Parcela'].astype(int)
dados['Vencimento'] = pd.to_datetime(dados['Vencimento'], format='%d/%m/%Y')
dados['Liquida o'] = pd.to_datetime(dados['Liquida o'], format='%d/%m/%Y')
dados.loc[:,'Recebido'] = dados.loc[:,'Recebido'].apply(dwsemear.converte_latin_para_float)
dados = dwsemear.liquidacoes_csv.separa_codigo_do_contrato(dados)
codigo_descricao = dwsemear.liquidacoes_csv.separa_forma_liquidacao(dados)
codigo_da_forma_de_liquidacao, descricao_da_forma_de_liquidacao = codigo_descricao
dados = dwsemear.liquidacoes_csv.separa_codigo_produto(dados)

In [9]:
liquidacoes_registros = []
id_liquidacao = -1
for i in dados.index:
    id_liquidacao += 1
    Data_de_liquidacao = dados.loc[i, 'Liquida o']
    Data_de_vencimento_da_parcela_liquidada = dados.loc[i, 'Vencimento']
    Numero_do_contrato = int(dados.loc[i,'codigos_contratos'])
    Codigo_renegociacao_contrato = int(dados.loc[i,'codigos_renegociacoes'])
    Parcela = int(dados.loc[i, 'Parcela'])
    Codigo_do_produto = int(dados.loc[i, 'produtos_codigos'])
    Numero_do_CPF = int(dados.loc[i,'CPF CNPJ'])
    Dias_em_atraso_na_hora_da_liquidacao = int(dados.loc[i,'Dias'])
    Valor_recebido_na_liquidacao = float(dados.loc[i,'Recebido'])
    Codigo_forma = codigo_da_forma_de_liquidacao[i]
    Descricao_forma = descricao_da_forma_de_liquidacao[i]
    Estornado = dados.loc[i,'Estornado']
    Situacao_da_liquidacao = dados.loc[i,'Situa o']
    liquidacoes_registros.append((
        id_liquidacao,
        Data_de_liquidacao,
        Data_de_vencimento_da_parcela_liquidada,
        Numero_do_CPF,
        Numero_do_contrato,
        Codigo_renegociacao_contrato,
        Codigo_do_produto,
        Parcela,
        Dias_em_atraso_na_hora_da_liquidacao,
        Valor_recebido_na_liquidacao,
        Codigo_forma,
        Situacao_da_liquidacao,
        Descricao_forma,
        Estornado))
    

# Carga no SQL

In [11]:
delete_tabela = '''DROP TABLE liquidacoes'''
sql.execute_query(delete_tabela)

In [12]:
crie_tabela = '''CREATE TABLE liquidacoes
            (ID BIGINT PRIMARY KEY NOT NULL,
            Data_de_liquidacao timestamp with time zone NOT NULL,
            Data_de_vencimento_da_parcela_liquidada timestamp with time zone NOT NULL,
            CPF BIGINT NOT NULL,
            Numero_do_contrato INT NOT NULL,
            Codigo_renegociacao_contrato SMALLINT,
            Codigo_do_produto SMALLINT,
            Parcela INT,
            Dias_em_atraso_na_hora_da_liquidacao INT,
            Valor_recebido_na_liquidacao REAL,
            Codigo_forma INT,
            Situacao_da_liquidacao TEXT,
            Descricao_forma TEXT,
            Estornado TEXT);'''
sql.execute_query(crie_tabela)

In [16]:
query = """ INSERT INTO liquidacoes
            (ID,
            Data_de_liquidacao,
            Data_de_vencimento_da_parcela_liquidada,
            CPF,
            Numero_do_contrato,
            Codigo_renegociacao_contrato,
            Codigo_do_produto,
            Parcela,
            Dias_em_atraso_na_hora_da_liquidacao,
            Valor_recebido_na_liquidacao,
            Codigo_forma,
            Situacao_da_liquidacao,
            Descricao_forma,
            Estornado) 
            VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"""
sql.adicione_multiplos_registros(query, liquidacoes_registros)

In [108]:
# os dados dessas querys nao vem ordenados
parametros = (datetime.datetime(2019,1,1), datetime.datetime(2019,2,1))
dados_liquidacao = sql.query_para_pandas('''SELECT *
                      FROM liquidacoes
                      WHERE Data_de_liquidacao BETWEEN %s AND %s
                      ''',parametros)

In [17]:
# os dados dessas querys nao vem ordenados
parametros = (0, 5)
dados_liquidacao = sql.query_para_pandas('''SELECT *
                      FROM liquidacoes
                      WHERE ID BETWEEN %s AND %s
                      ''',parametros)